# Clustering Neighborhoods in Toronto
## By Mathias Hochkofler
### Part 1

#### Import librarys

In [4]:
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#### Create Data Frame of Toronto Postal Codes

In [6]:
# Scrape the Wikipedia link in order to obtain the DataFrame 
toronto_data = pd.DataFrame(pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0])
# Filter 'Not assigned' Boroughs
toronto_data = toronto_data[toronto_data.Borough != 'Not assigned']
# Replace blank neighborhoods to Botough column
toronto_data['Neighborhood'] = toronto_data.Neighborhood.replace(r'^\s*$',toronto_data.Borough, regex=True)

toronto_data.shape



(103, 3)

### Part 2
#### Get Geographocal coordinates of the neighborhoods

In [7]:
# I´m not able to get the geographical coordinates of the neighborhoods using the Geocoder package
geo_coordinates = pd.read_csv('https://cocl.us/Geospatial_data')

In [8]:
# Merge Two columns using Postal Code as key
tor_data = pd.merge(toronto_data, geo_coordinates, on=['Postal Code', 'Postal Code'])
tor_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Part 3
#### Use geopy library to get the latitude and longitude values of Toronto

In [9]:
from geopy.geocoders import Nominatim
address = 'Toronto, TOR'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.7627912, -79.4064452.


In [10]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(tor_data['Latitude'], tor_data['Longitude'], tor_data['Borough'], tor_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

#### we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them of Toronto.
#### Define Foursquare Credentials and Version

In [11]:
CLIENT_ID = 'LIXHVJHCNOPQPGVE4J2FJJXWLCAHK40KB3V5GH1TRDYUGAMD' # your Foursquare ID
CLIENT_SECRET = 'XAX0EAA2ATNV3NPW105XQALDOZ2QFK2URBDSXXALTTMPLBYV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: LIXHVJHCNOPQPGVE4J2FJJXWLCAHK40KB3V5GH1TRDYUGAMD
CLIENT_SECRET:XAX0EAA2ATNV3NPW105XQALDOZ2QFK2URBDSXXALTTMPLBYV


## Part 5
### Explore Neighborhoods in Toronto
#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [13]:
toronto_venues = getNearbyVenues(names=tor_data['Neighborhood'],
                                   latitudes=tor_data['Latitude'],
                                   longitudes=tor_data['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

Let's check how many venues were returned for each neighborhood

In [21]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Wilson Heights, Downsview North",23,23,23,23,23,23
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Berczy Park,55,55,55,55,55,55
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",18,18,18,18,18,18


## Part 4
### Analyze Each Neighborhood

In [40]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# Delete Neighborhood column wich is empty
del toronto_onehot['Neighborhood']
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [41]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,"Alderwood, Long Branch",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
5,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
6,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
7,"Brockton, Parkdale Village, Exhibition Place",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
8,"Business reply mail Processing Centre, South C...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.055556
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.000000,0.071429,0.071429,0.071429,0.142857,0.071429,0.142857,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


In [42]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25
3               Skating Rink  0.25
4         Mexican Restaurant  0.00


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.22
1      Pharmacy  0.11
2           Gym  0.11
3  Dance Studio  0.11
4  Skating Rink  0.11


----Bathurst Manor, Wilson Heights, Downsview North----
               venue  freq
0               Bank  0.09
1        Coffee Shop  0.09
2  Convenience Store  0.04
3      Deli / Bodega  0.04
4      Shopping Mall  0.04


----Bayview Village----
                 venue  freq
0                 Café  0.25
1   Chinese Restaurant  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4          Men's Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.08
1          Restaurant  0.08
2  Italian Restaurant  0.08
3         Coffee Shop  0.08
4  

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [131]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [138]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Latin American Restaurant,Breakfast Spot,Doner Restaurant
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Sandwich Place,Pub,Skating Rink
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Diner,Frozen Yogurt Shop,Middle Eastern Restaurant
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Restaurant,Thai Restaurant


## Part 6
### Cluster Neighborhoods

In [139]:
# set number of clusters
kclusters = 5

toronoto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronoto_grouped_clustering)

# check cluster labels generated for each row in the dataframe



In [140]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,1,Agincourt,Lounge,Skating Rink,Latin American Restaurant,Breakfast Spot,Doner Restaurant
1,0,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Sandwich Place,Pub,Skating Rink
2,1,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Diner,Frozen Yogurt Shop,Middle Eastern Restaurant
3,1,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Yoga Studio
4,1,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Restaurant,Thai Restaurant


In [141]:
tor_data.shape

(103, 5)

In [142]:
toronto_merged = tor_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Park,Food & Drink Shop,Yoga Studio,Dog Run,Dessert Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,French Restaurant,Intersection,Pizza Place,Coffee Shop,Portuguese Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Pub,Bakery,Park,Café
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Furniture / Home Store,Clothing Store,Accessories Store,Gift Shop,Boutique
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Yoga Studio,Arts & Crafts Store,Diner,Bar
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1.0,Print Shop,Fast Food Restaurant,Dog Run,Department Store,Dessert Shop
7,M3B,North York,Don Mills,43.745906,-79.352188,1.0,Coffee Shop,Restaurant,Gym,Japanese Restaurant,Beer Store
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1.0,Pizza Place,Gym / Fitness Center,Athletics & Sports,Gastropub,Intersection
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1.0,Clothing Store,Coffee Shop,Japanese Restaurant,Bubble Tea Shop,Café


In [143]:
# Drop NA from toronto_marged
toronto_merged= toronto_merged[toronto_merged['Cluster Labels'].notna()]
toronto_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Park,Food & Drink Shop,Yoga Studio,Dog Run,Dessert Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,French Restaurant,Intersection,Pizza Place,Coffee Shop,Portuguese Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Pub,Bakery,Park,Café
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Furniture / Home Store,Clothing Store,Accessories Store,Gift Shop,Boutique
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Yoga Studio,Arts & Crafts Store,Diner,Bar
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1.0,Print Shop,Fast Food Restaurant,Dog Run,Department Store,Dessert Shop
7,M3B,North York,Don Mills,43.745906,-79.352188,1.0,Coffee Shop,Restaurant,Gym,Japanese Restaurant,Beer Store
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1.0,Pizza Place,Gym / Fitness Center,Athletics & Sports,Gastropub,Intersection
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1.0,Clothing Store,Coffee Shop,Japanese Restaurant,Bubble Tea Shop,Café
10,M6B,North York,Glencairn,43.709577,-79.445073,0.0,Pizza Place,Park,Sushi Restaurant,Pub,Japanese Restaurant


In [146]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Part 7
### Examine Clusters

#### Cluster 1

In [149]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,North York,0.0,French Restaurant,Intersection,Pizza Place,Coffee Shop,Portuguese Restaurant
10,North York,0.0,Pizza Place,Park,Sushi Restaurant,Pub,Japanese Restaurant
50,North York,0.0,Pizza Place,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant
70,Etobicoke,0.0,Pizza Place,Chinese Restaurant,Intersection,Coffee Shop,Sandwich Place
77,Etobicoke,0.0,Pizza Place,Mobile Phone Shop,Bus Line,Sandwich Place,Yoga Studio
93,Etobicoke,0.0,Pizza Place,Coffee Shop,Sandwich Place,Pub,Skating Rink


#### Cluster 2

In [150]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Downtown Toronto,1.0,Coffee Shop,Pub,Bakery,Park,Café
3,North York,1.0,Furniture / Home Store,Clothing Store,Accessories Store,Gift Shop,Boutique
4,Downtown Toronto,1.0,Coffee Shop,Yoga Studio,Arts & Crafts Store,Diner,Bar
6,Scarborough,1.0,Print Shop,Fast Food Restaurant,Dog Run,Department Store,Dessert Shop
7,North York,1.0,Coffee Shop,Restaurant,Gym,Japanese Restaurant,Beer Store
8,East York,1.0,Pizza Place,Gym / Fitness Center,Athletics & Sports,Gastropub,Intersection
9,Downtown Toronto,1.0,Clothing Store,Coffee Shop,Japanese Restaurant,Bubble Tea Shop,Café
12,Scarborough,1.0,Bar,History Museum,Yoga Studio,Dog Run,Dessert Shop
13,North York,1.0,Coffee Shop,Restaurant,Gym,Japanese Restaurant,Beer Store
14,East York,1.0,Pharmacy,Video Store,Curling Ice,Diner,Cosmetics Shop


#### Cluster 3

In [151]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
64,York,2.0,Convenience Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store


#### Cluster 4

In [152]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,North York,3.0,Park,Food & Drink Shop,Yoga Studio,Dog Run,Dessert Shop
21,York,3.0,Park,Women's Store,Pool,Dog Run,Department Store
35,East York,3.0,Park,Convenience Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant
61,Central Toronto,3.0,Park,Bus Line,Swim School,Yoga Studio,Distribution Center
66,North York,3.0,Convenience Store,Park,Bank,Doner Restaurant,Dim Sum Restaurant
68,Central Toronto,3.0,Park,Sushi Restaurant,Jewelry Store,Trail,Distribution Center
85,Scarborough,3.0,Playground,Park,Yoga Studio,Dog Run,Department Store
91,Downtown Toronto,3.0,Park,Trail,Playground,Yoga Studio,Deli / Bodega


#### Cluster 5

In [153]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
57,North York,4.0,Baseball Field,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner
